In [1]:
from tqdm import tqdm
import json

In [2]:
# 主要是对运行equal_split_word.py之后的处理流程
# 包括处理word_dict，得到vocab，生成vocab_split，生成nofeat版本的数据

In [3]:
# 读入已处理的词表
with open('../vocab/vocab_dict.json', 'r') as f:
    vocab_dict = json.load(f)

In [4]:
# 读入未处理的词表
with open('../vocab/word_dict.json', 'r') as f:
    word_dict = json.load(f)

In [5]:
# 统一大量颜色的别称
color_list = ['兰','蓝','灰','绿','粉','红','黄','青','紫','白','黑','驼','橙','杏','咖','棕','啡','褐','银','金','橘','藏']
word_list = list(word_dict.keys())

def union_word_dict(word_dict, new_word, ori_word):
    if new_word in word_dict:
        word_dict[new_word] += word_dict[ori_word]
    else:
        word_dict[new_word] = word_dict[ori_word]

for word in word_list:
    if word_dict[word] < 50: # 注意这个值要与后面删除的阈值保持一致
        # 两个特殊的颜色
        rep_word = word
        if '卡其' in rep_word:
            union_word_dict(word_dict, '卡其', word)
            rep_word = rep_word.replace('卡其', '')
        if '咖啡' in rep_word:
            union_word_dict(word_dict, '咖啡', word)
            rep_word = rep_word.replace('咖啡', '')
        for char in rep_word:
            if char in color_list:
                union_word_dict(word_dict, char, word)

In [6]:
# 读入属性词表
def get_dict(file):
    with open(file, 'r') as f:
        all_attr = []
        for attr, attrval_list in json.load(f).items():
            for x in attrval_list:
                all_attr.append(x)
    return all_attr
attr_dict_file = '../../data/equal_processed_data/attr_to_attrvals.json'
all_attr = get_dict(attr_dict_file)

# 删除出现次数少的词，不删除属性词，不删除我们设置的颜色值（否则后面可能会不匹配出现bug）
ignore_list = all_attr + color_list + ['卡其', '咖啡']
word_list = list(word_dict.keys())
for word in word_list:
    if word_dict[word] < 50 and word not in ignore_list:
        del word_dict[word]
    if word == '/':
        del word_dict[word]
        
# 去掉没有单独意义的字词
delete_words = ['色','小','本','中','新','款','加','底','件','不']
for word in delete_words:
    del word_dict[word]
    
# 保存处理后的词表
vocab_dict = word_dict
with open('../vocab/vocab_dict.json', 'w') as f:
    json.dump(vocab_dict, f, ensure_ascii=False)

In [7]:
# [fine] 根据处理后的词表对数据的分词进行筛选
fine_file = '../../data/split_word/fine45000.txt'
save_fine_file = '../../data/equal_split_word/fine45000.txt'

vocab_list = list(vocab_dict.keys())
color_list = ['兰','蓝','灰','绿','粉','红','黄','青','紫','白','黑','驼','橙','杏','咖','棕','啡','褐','银','金','橘','藏']
mul_color_list = ['卡其', '咖啡']
rets = []

with open(fine_file, 'r') as f:
    for i, line in enumerate(tqdm(f)):
        item = json.loads(line)
        title_split = item['title_split']
        
        vocab_split = []
        for word in title_split:
            if word in vocab_list:
                vocab_split.append(word)
            else: # 颜色提取
                rep_word = word
                for mul_color in mul_color_list:
                    if mul_color in rep_word:
                        vocab_split.append(mul_color)
                        rep_word = rep_word.replace(mul_color, '')
                for char in rep_word:
                    if char in color_list:
                        vocab_split.append(char)
        item['vocab_split'] = vocab_split
        # 看看有没有空的vocab_split
        if not vocab_split:
            print(item['title'])
        # 更改保存的顺序，便于查看
        feature = item['feature']
        del item['feature']
        item['feature'] = feature
        
        rets.append(json.dumps(item, ensure_ascii=False)+'\n')
        
        # if i>500:
        #     break
        # i += 1
        
with open(save_fine_file, 'w') as f:
    f.writelines(rets)

45000it [00:49, 904.70it/s]


In [8]:
# [coarse] 根据处理后的词表对数据的分词进行筛选
coarse_file = '../../data/split_word/coarse89588.txt'
save_coarse_file = '../../data/equal_split_word/coarse89588.txt'

vocab_list = list(vocab_dict.keys())
color_list = ['兰','蓝','灰','绿','粉','红','黄','青','紫','白','黑','驼','橙','杏','咖','棕','啡','褐','银','金','橘','藏']
mul_color_list = ['卡其', '咖啡']
rets = []

with open(coarse_file, 'r') as f:
    for i, line in enumerate(tqdm(f)):
        item = json.loads(line)
        title_split = item['title_split']
        
        vocab_split = []
        for word in title_split:
            if word in vocab_list:
                vocab_split.append(word)
            else: # 颜色提取
                rep_word = word
                for mul_color in mul_color_list:
                    if mul_color in rep_word:
                        vocab_split.append(mul_color)
                        rep_word = rep_word.replace(mul_color, '')
                for char in rep_word:
                    if char in color_list:
                        vocab_split.append(char)
        item['vocab_split'] = vocab_split
        # 看看有没有空的vocab_split
        if not vocab_split:
            print(item['title'])
        # 更改保存的顺序，便于查看
        feature = item['feature']
        del item['feature']
        item['feature'] = feature
        
        rets.append(json.dumps(item, ensure_ascii=False)+'\n')
        
        # if i>500:
        #     break
        # i += 1
        
with open(save_coarse_file, 'w') as f:
    f.writelines(rets)

89588it [01:39, 900.90it/s]


In [9]:
# [fine]生成nofeat版本
fine_file = '../../data/equal_split_word/fine45000.txt'
save_fine_file = '../../data/equal_split_word/nofeat/fine45000.txt'

rets = []
with open(fine_file, 'r') as f:
    for i, line in enumerate(tqdm(f)):
        item = json.loads(line)
        del item['feature']
        rets.append(json.dumps(item, ensure_ascii=False)+'\n')
        
with open(save_fine_file, 'w') as f:
    f.writelines(rets)

45000it [00:12, 3573.45it/s]


In [10]:
# [coarse]生成nofeat版本
coarse_file = '../../data/equal_split_word/coarse89588.txt'
save_coarse_file = '../../data/equal_split_word/nofeat/coarse89588.txt'

rets = []
with open(coarse_file, 'r') as f:
    for i, line in enumerate(tqdm(f)):
        item = json.loads(line)
        del item['feature']
        rets.append(json.dumps(item, ensure_ascii=False)+'\n')
        
with open(save_coarse_file, 'w') as f:
    f.writelines(rets)

89588it [00:25, 3528.09it/s]


In [11]:
# [fine5000] 根据处理后的词表对数据的分词进行筛选
fine_file = '../../data/split_word/fine5000.txt'
save_fine_file = '../../data/equal_split_word/fine5000.txt'

vocab_list = list(vocab_dict.keys())
color_list = ['兰','蓝','灰','绿','粉','红','黄','青','紫','白','黑','驼','橙','杏','咖','棕','啡','褐','银','金','橘','藏']
mul_color_list = ['卡其', '咖啡']
rets = []

with open(fine_file, 'r') as f:
    for i, line in enumerate(tqdm(f)):
        item = json.loads(line)
        title_split = item['title_split']
        
        vocab_split = []
        for word in title_split:
            if word in vocab_list:
                vocab_split.append(word)
            else: # 颜色提取
                rep_word = word
                for mul_color in mul_color_list:
                    if mul_color in rep_word:
                        vocab_split.append(mul_color)
                        rep_word = rep_word.replace(mul_color, '')
                for char in rep_word:
                    if char in color_list:
                        vocab_split.append(char)
        item['vocab_split'] = vocab_split
        # 看看有没有空的vocab_split
        if not vocab_split:
            print(item['title'])
        # 更改保存的顺序，便于查看
        feature = item['feature']
        del item['feature']
        item['feature'] = feature
        
        rets.append(json.dumps(item, ensure_ascii=False)+'\n')
        
        # if i>500:
        #     break
        # i += 1
        
with open(save_fine_file, 'w') as f:
    f.writelines(rets)

5000it [00:05, 886.22it/s]


In [12]:
# [coarse10412] 根据处理后的词表对数据的分词进行筛选
coarse_file = '../../data/split_word/coarse10412.txt'
save_coarse_file = '../../data/equal_split_word/coarse10412.txt'

vocab_list = list(vocab_dict.keys())
color_list = ['兰','蓝','灰','绿','粉','红','黄','青','紫','白','黑','驼','橙','杏','咖','棕','啡','褐','银','金','橘','藏']
mul_color_list = ['卡其', '咖啡']
rets = []

with open(coarse_file, 'r') as f:
    for i, line in enumerate(tqdm(f)):
        item = json.loads(line)
        title_split = item['title_split']
        
        vocab_split = []
        for word in title_split:
            if word in vocab_list:
                vocab_split.append(word)
            else: # 颜色提取
                rep_word = word
                for mul_color in mul_color_list:
                    if mul_color in rep_word:
                        vocab_split.append(mul_color)
                        rep_word = rep_word.replace(mul_color, '')
                for char in rep_word:
                    if char in color_list:
                        vocab_split.append(char)
        item['vocab_split'] = vocab_split
        # 看看有没有空的vocab_split
        if not vocab_split:
            print(item['title'])
        # 更改保存的顺序，便于查看
        feature = item['feature']
        del item['feature']
        item['feature'] = feature
        
        rets.append(json.dumps(item, ensure_ascii=False)+'\n')
        
        # if i>500:
        #     break
        # i += 1
        
with open(save_coarse_file, 'w') as f:
    f.writelines(rets)

10412it [00:11, 904.20it/s]


In [13]:
# [fine5000]生成nofeat版本
fine_file = '../../data/equal_split_word/fine5000.txt'
save_fine_file = '../../data/equal_split_word/nofeat/fine5000.txt'

rets = []
with open(fine_file, 'r') as f:
    for i, line in enumerate(tqdm(f)):
        item = json.loads(line)
        del item['feature']
        rets.append(json.dumps(item, ensure_ascii=False)+'\n')
        
with open(save_fine_file, 'w') as f:
    f.writelines(rets)

5000it [00:01, 3539.21it/s]


In [14]:
# [coarse10412]生成nofeat版本
coarse_file = '../../data/equal_split_word/coarse10412.txt'
save_coarse_file = '../../data/equal_split_word/nofeat/coarse10412.txt'

rets = []
with open(coarse_file, 'r') as f:
    for i, line in enumerate(tqdm(f)):
        item = json.loads(line)
        del item['feature']
        rets.append(json.dumps(item, ensure_ascii=False)+'\n')
        
with open(save_coarse_file, 'w') as f:
    f.writelines(rets)

10412it [00:02, 3619.07it/s]


In [5]:
# [test] 根据处理后的词表对数据的分词进行筛选
coarse_file = '../../data/split_word/test10000.txt'
save_coarse_file = '../../data/equal_split_word/test10000.txt'

vocab_list = list(vocab_dict.keys())
color_list = ['兰','蓝','灰','绿','粉','红','黄','青','紫','白','黑','驼','橙','杏','咖','棕','啡','褐','银','金','橘','藏']
mul_color_list = ['卡其', '咖啡']
rets = []

with open(coarse_file, 'r') as f:
    for i, line in enumerate(tqdm(f)):
        item = json.loads(line)
        title_split = item['title_split']
        
        vocab_split = []
        for word in title_split:
            if word in vocab_list:
                vocab_split.append(word)
            else: # 颜色提取
                rep_word = word
                for mul_color in mul_color_list:
                    if mul_color in rep_word:
                        vocab_split.append(mul_color)
                        rep_word = rep_word.replace(mul_color, '')
                for char in rep_word:
                    if char in color_list:
                        vocab_split.append(char)
        item['vocab_split'] = vocab_split
        # 看看有没有空的vocab_split
        if not vocab_split:
            print(item['title'])
        # 更改保存的顺序，便于查看
        feature = item['feature']
        del item['feature']
        item['feature'] = feature
        
        rets.append(json.dumps(item, ensure_ascii=False)+'\n')
        
        # if i>500:
        #     break
        # i += 1
        
with open(save_coarse_file, 'w') as f:
    f.writelines(rets)

10000it [00:11, 893.23it/s]


In [6]:
# [test]生成nofeat版本
file = '../../data/equal_split_word/test10000.txt'
save_file = '../../data/equal_split_word/nofeat/test10000.txt'

rets = []
with open(file, 'r') as f:
    for i, line in enumerate(tqdm(f)):
        item = json.loads(line)
        del item['feature']
        rets.append(json.dumps(item, ensure_ascii=False)+'\n')
        
with open(save_file, 'w') as f:
    f.writelines(rets)

10000it [00:02, 3564.92it/s]
